<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/main/XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optional

In [ ]:
!pip uninstall -y scikit-learn xgboost
!pip install scikit-learn==1.2.2 xgboost==1.7.6

Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Successfully uninstalled scikit-learn-1.6.0
Found existing installation: xgboost 2.1.3
Uninstalling xgboost-2.1.3:
  Successfully uninstalled xgboost-2.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


Importing Required Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import zipfile
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pandas as pd

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Loading Supervised Datasets from GitHub Repository

In [2]:
import zipfile
import requests
from io import BytesIO

# Define the base URL of the repository and dataset file names
base_url = "https://github.com/orifelszer/CrimeData/raw/main/"
files = {
    "X_train": "X_train_supervised.zip",
    "X_test": "X_test_supervised.zip",
    "y_train": "y_train_supervised.zip",
    "y_test": "y_test_supervised.zip"}

# Function to download and extract ZIP files from the GitHub repository
def load_data_from_repo(file_name):
    url = base_url + file_name
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            extracted_file_name = z.namelist()[0]
            with z.open(extracted_file_name) as f:
                return pd.read_csv(f)
    else:
        raise Exception(f"Failed to download {file_name}")

# Load datasets from the repository
X_train = pd.DataFrame(load_data_from_repo(files["X_train"]))
X_test = pd.DataFrame(load_data_from_repo(files["X_test"]))
y_train = pd.DataFrame(load_data_from_repo(files["y_train"]))
y_test = pd.DataFrame(load_data_from_repo(files["y_test"]))

Defining the XGBoost Classifier with Hyperparameters

In [ ]:
# Initialize the XGBoost model with specified hyperparameters
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    random_state=42)

Training the XGBoost Classifier

In [ ]:
# Train the XGBoost model using the training data
xgb_model.fit(X_train.values, y_train.values)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

Making Predictions with the Trained XGBoost Classifier

In [ ]:
# Generate predictions on the test set using the trained XGBoost model
y_pred = xgb_model.predict(X_test)

Evaluation Function for Classification Models

In [ ]:
# Define a function to evaluate a classification model using multiple metrics
def evaluate_model(y_true, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "F1 Score": f1_score(y_true, y_pred, average="weighted", zero_division=0)
    }

Printing the Performance of the XGBoost Model

In [ ]:
# Evaluate the XGBoost model using the custom evaluation function
xgb_metrics = evaluate_model(y_test, y_pred)

print("Improved XGBoost Metrics:", xgb_metrics)

Improved XGBoost Metrics: {'Accuracy': 0.3821063158704516, 'Precision': 0.35156940510210927, 'Recall': 0.3821063158704516, 'F1 Score': 0.3136799844272996}
